In [5]:
# example of a dcgan 


In [6]:
import os
import tensorflow as tf
from keras.utils import plot_model
import pydot
import graphviz
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from numpy import expand_dims, zeros, ones, vstack
from numpy.random import randn, randint
from keras.optimizers import Adam, SGD
from keras.models import Sequential  
from keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU, Dropout, BatchNormalization  
from matplotlib import pyplot
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
from cv2 import cv2
from PIL import Image

In [3]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [4]:
# we will select images with label 4 since it is a minority class
df_csv = pd.read_csv('/storage/trainLabels.csv')
df_csv['image'] = df_csv['image'].astype(str) + '.jpeg'

df_txt = pd.read_table('/storage/name.txt',  header=None)
df_txt.columns = ['name']

df = df_csv[df_csv.image.isin(df_txt.name.values)]
len(df[df['level'] == 4])

643

In [5]:
df_4 = df[df['level'] == 4]

In [6]:
df_4.head()

,image,level
6,16_left.jpeg,4
7,16_right.jpeg,4
184,217_left.jpeg,4
185,217_right.jpeg,4
252,294_left.jpeg,4


In [7]:
# define the standalone discriminator model using GAN training hacks 
def define_discriminator(in_shape=(128,128,3)):
    model = Sequential()
    # input layer with image size of 128x128, since its a colored image it has 3 channels
    model.add(Conv2D(16, (3,3), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    # downsample to 64x64 using strides of 2,2 and use of LeakyReLU
    model.add(Conv2D(8, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # downsample to 32x32
    model.add(Conv2D(16, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # downsample to 16x16
    model.add(Conv2D(8, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # now the image size is down to 16 x 16
    # classifier 
    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    # compile model learning rate is higher than generator 2e-3
    # use adam optimizer 
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model
 
# define the standalone generator model using GAN training hacks
def define_generator(latent_dim):
    model = Sequential()
    # foundation for 16x16 image
    n_nodes = 256 * 16 * 16
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((16, 16, 256)))
    # upsample to 32x32, use of strides and LeakyReLU
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 64x64
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 128x128
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # output layer, use of tanh as per hacks
    model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))
    return model
 

In [8]:
# combined generator and discriminator model, for updating the generator
# this is alogical GAN model using above defined generator and discriminator 
def define_gan(g_model, d_model):
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # initialize with a sequential model
    model = Sequential()
    # generator and discriminator are initialized later in the code to g_model and d_model respectively 
    # add the generator
    model.add(g_model)
    # add the discriminator
    model.add(d_model)
    # compile model
    # use of adam optimizer, learning rate is lower than discriminator 2e-4
    opt = Adam(lr=0.00002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

In [9]:
# read images from df_4 dataframe into x_train
x_train = []
def load_real_samples():
    for f, breed in tqdm(df_4.values):
        try:
            img = image.load_img(('/storage/train/{}'.format(f)), target_size=(128, 128))
            # convert to float32
            arr1 = image.img_to_array(img, dtype = 'float32')
            # scale images to [-1,1] from [0,255]
            arr = (arr1 - 127.5) / 127.5
            x_train.append(arr)
        except:
            pass
    return x_train

In [13]:
# select real samples
# images are loaded into dataset variable using load_real_samples() function
# below function will randomly select images from dataset and spit out X and y 
# X contains images, y contains lables. (model is not trained with labels as all the images are of same label)
def generate_real_samples(dataset, n_samples):
    # choose random instances
    ix = randint(0, len(dataset), n_samples)
    # generate 'real' class labels (1)
    y = ones((n_samples, 1))   
    # retrieve selected images
    X = []  
    f = 0 
    for f in range (len(ix)):
        X.append(dataset[ix[f]])
    #print(len(X))
    return X, y   

In [15]:
# Use Gaussian Latent Space
# generate points in latent space as input for the generator
# The latent space defines the shape and distribution of the input to the generator model used to generate new images.
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input
 
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
	# generate points in latent space
	x_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	X = g_model.predict(x_input)
	# create 'fake' class labels (0)
	y = zeros((n_samples, 1))
	return X, y

In [16]:
# create and save a plot of generated images
def save_plot(examples, epoch, n=7):
	# scale from [-1,1] to [0,1]
	examples = (examples + 1) / 2.0
	# plot images
	for i in range(n * n):
		# define subplot
		pyplot.subplot(n, n, 1 + i)
		# turn off axis
		pyplot.axis('off')
		# plot raw pixel data
		pyplot.imshow(examples[i])
	# save plot to file
	filename = 'generated_plot_e%03d.png' % (epoch+1)
	pyplot.savefig(filename)
	pyplot.close()
 
# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=49):
    print("################# Summarize ###################")
    # prepare real samples
    X_real, y_real = generate_real_samples(dataset, n_samples)
    # evaluate discriminator on real examples
    X_real_raw = np.array(X_real)
    y_real_raw = np.array(y_real)
    _, acc_real = d_model.evaluate(X_real_raw, y_real_raw, verbose=0)
    # prepare fake examples
    x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
    # evaluate discriminator on fake examples
    _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
    # summarize discriminator performance
    print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
    # save plot
    save_plot(x_fake, epoch)
    # save the generator model tile file
    filename = 'generator_model_%03d.h5' % (epoch+1)
    g_model.save(filename)

In [17]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=320, n_batch=64):
    bat_per_epo = int(len(dataset) / n_batch)
    #print('Batches per Epoch is %d' %bat_per_epo)
    half_batch = int(n_batch / 2)
    #print("Half Batch %d" % half_batch)
    # manually enumerate epochs
    for i in range(n_epochs):
        # enumerate batches over the training set
        for j in range(bat_per_epo):
            # print("Batch number %d" %(j+1))
            # get randomly selected 'real' samples
            X_real, y_real = generate_real_samples(dataset, half_batch)
            #print(len(X_real))  
            X_real_raw = np.array(X_real)
            y_real_raw = np.array(y_real)
            # update discriminator model weights with real images
            # It is reccomended to update discriminator with seperate batches of real and fake images
            # update discriminator model with real images 
            d_loss1, _ = d_model.train_on_batch(X_real_raw, y_real_raw)
            # generate 'fake' examples
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            # update discriminator model with fake images         
            d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)
            # prepare points in latent space as input for the generator
            X_gan = generate_latent_points(latent_dim, n_batch)
            # create inverted labels for the fake samples
            y_gan = ones((n_batch, 1))
            # update the generator via the discriminator's error
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            # summarize loss on this batch
            print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
                (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
        # evaluate the model performance, sometimes
        if (i+1) % 10 == 0:
            summarize_performance(i, g_model, d_model, dataset, latent_dim)
 

In [18]:
# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)

In [19]:
plot_model( d_model, to_file='Discriminator.png', show_shapes=True, show_layer_names=True, rankdir='TB')
plot_model( g_model, to_file='Generator.png', show_shapes=True, show_layer_names=True, rankdir='TB')

In [20]:
# load image data
dataset = load_real_samples()
# train model

100%|██████████| 643/643 [01:22<00:00,  7.92it/s]


In [22]:
d_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 128, 128, 16)      448       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 128, 128, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 8)         1160      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 64, 64, 8)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 16)        1168      
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 32, 32, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 8)         1160      
__________

/usr/local/lib/python3.5/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


In [23]:
g_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 65536)             6619136   
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 65536)             0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 32, 32, 128)       524416    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 32, 32, 128)       0         
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 64, 64, 128)       262272    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 64, 64, 128)       0         
__________

In [24]:
with tf.device('/device:GPU:0'):
    train(g_model, d_model, gan_model, dataset, latent_dim)

/usr/local/lib/python3.5/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


>1, 1/10, d1=0.779, d2=0.694 g=0.693
>1, 2/10, d1=0.749, d2=0.694 g=0.693
>1, 3/10, d1=0.720, d2=0.694 g=0.693
>1, 4/10, d1=0.690, d2=0.694 g=0.693
>1, 5/10, d1=0.650, d2=0.694 g=0.692
>1, 6/10, d1=0.623, d2=0.694 g=0.692
>1, 7/10, d1=0.604, d2=0.694 g=0.692
>1, 8/10, d1=0.548, d2=0.695 g=0.692
>1, 9/10, d1=0.521, d2=0.695 g=0.692
>1, 10/10, d1=0.478, d2=0.695 g=0.691
>2, 1/10, d1=0.428, d2=0.696 g=0.692
>2, 2/10, d1=0.399, d2=0.696 g=0.692
>2, 3/10, d1=0.323, d2=0.695 g=0.692
>2, 4/10, d1=0.282, d2=0.695 g=0.693
>2, 5/10, d1=0.200, d2=0.694 g=0.694
>2, 6/10, d1=0.179, d2=0.693 g=0.695
>2, 7/10, d1=0.138, d2=0.692 g=0.695
>2, 8/10, d1=0.111, d2=0.691 g=0.696
>2, 9/10, d1=0.091, d2=0.690 g=0.698
>2, 10/10, d1=0.090, d2=0.689 g=0.700
>3, 1/10, d1=0.066, d2=0.686 g=0.702
>3, 2/10, d1=0.052, d2=0.685 g=0.703
>3, 3/10, d1=0.051, d2=0.683 g=0.705
>3, 4/10, d1=0.036, d2=0.680 g=0.708
>3, 5/10, d1=0.035, d2=0.678 g=0.710
>3, 6/10, d1=0.027, d2=0.676 g=0.713
>3, 7/10, d1=0.032, d2=0.673 g=0.716

>22, 5/10, d1=0.031, d2=0.001 g=6.815
>22, 6/10, d1=0.002, d2=0.012 g=6.006
>22, 7/10, d1=0.000, d2=0.010 g=6.792
>22, 8/10, d1=0.000, d2=0.003 g=6.837
>22, 9/10, d1=0.007, d2=0.009 g=6.357
>22, 10/10, d1=0.007, d2=0.012 g=7.149
>23, 1/10, d1=0.006, d2=0.003 g=6.955
>23, 2/10, d1=0.008, d2=0.013 g=7.117
>23, 3/10, d1=0.010, d2=0.005 g=6.716
>23, 4/10, d1=0.004, d2=0.008 g=7.194
>23, 5/10, d1=0.023, d2=0.052 g=12.388
>23, 6/10, d1=0.486, d2=2.230 g=13.129
>23, 7/10, d1=0.123, d2=0.000 g=16.118
>23, 8/10, d1=3.952, d2=0.000 g=3.391
>23, 9/10, d1=0.000, d2=2.287 g=5.601
>23, 10/10, d1=0.000, d2=0.002 g=8.180
>24, 1/10, d1=0.002, d2=0.002 g=6.740
>24, 2/10, d1=0.002, d2=0.023 g=4.392
>24, 3/10, d1=0.014, d2=0.215 g=4.345
>24, 4/10, d1=0.030, d2=0.097 g=4.482
>24, 5/10, d1=0.015, d2=0.092 g=4.683
>24, 6/10, d1=0.131, d2=0.111 g=4.115
>24, 7/10, d1=0.304, d2=0.585 g=7.290
>24, 8/10, d1=1.315, d2=0.132 g=2.600
>24, 9/10, d1=0.151, d2=0.788 g=7.644
>24, 10/10, d1=1.494, d2=0.243 g=1.742
>25, 1

>43, 6/10, d1=0.163, d2=0.167 g=3.995
>43, 7/10, d1=0.374, d2=0.398 g=3.711
>43, 8/10, d1=0.614, d2=0.282 g=2.807
>43, 9/10, d1=0.191, d2=0.170 g=3.284
>43, 10/10, d1=0.222, d2=0.153 g=3.007
>44, 1/10, d1=0.083, d2=0.285 g=3.837
>44, 2/10, d1=0.380, d2=0.178 g=3.721
>44, 3/10, d1=0.379, d2=0.306 g=3.432
>44, 4/10, d1=0.240, d2=0.261 g=3.389
>44, 5/10, d1=0.377, d2=0.308 g=3.858
>44, 6/10, d1=0.144, d2=0.062 g=3.455
>44, 7/10, d1=0.249, d2=0.190 g=3.317
>44, 8/10, d1=0.120, d2=0.190 g=4.338
>44, 9/10, d1=0.266, d2=0.080 g=3.437
>44, 10/10, d1=0.165, d2=0.165 g=3.206
>45, 1/10, d1=0.111, d2=0.129 g=4.266
>45, 2/10, d1=0.391, d2=0.165 g=2.923
>45, 3/10, d1=0.286, d2=0.500 g=3.683
>45, 4/10, d1=0.246, d2=0.064 g=4.091
>45, 5/10, d1=0.446, d2=0.243 g=3.219
>45, 6/10, d1=0.122, d2=0.197 g=4.045
>45, 7/10, d1=0.331, d2=0.107 g=3.140
>45, 8/10, d1=0.076, d2=0.181 g=3.786
>45, 9/10, d1=0.384, d2=0.139 g=3.175
>45, 10/10, d1=0.055, d2=0.190 g=4.366
>46, 1/10, d1=0.929, d2=0.355 g=2.224
>46, 2/10

>64, 7/10, d1=0.560, d2=0.294 g=2.163
>64, 8/10, d1=0.372, d2=0.299 g=2.482
>64, 9/10, d1=0.282, d2=0.291 g=2.770
>64, 10/10, d1=0.557, d2=0.294 g=2.113
>65, 1/10, d1=0.417, d2=0.311 g=2.502
>65, 2/10, d1=0.210, d2=0.236 g=2.958
>65, 3/10, d1=0.589, d2=0.274 g=2.291
>65, 4/10, d1=0.326, d2=0.174 g=2.141
>65, 5/10, d1=0.369, d2=0.418 g=2.270
>65, 6/10, d1=0.561, d2=0.417 g=2.323
>65, 7/10, d1=0.293, d2=0.263 g=2.775
>65, 8/10, d1=0.561, d2=0.242 g=2.364
>65, 9/10, d1=0.520, d2=0.341 g=1.794
>65, 10/10, d1=0.209, d2=0.357 g=2.795
>66, 1/10, d1=0.660, d2=0.327 g=2.109
>66, 2/10, d1=0.495, d2=0.425 g=2.353
>66, 3/10, d1=0.331, d2=0.280 g=2.262
>66, 4/10, d1=0.643, d2=0.488 g=2.213
>66, 5/10, d1=0.209, d2=0.196 g=2.968
>66, 6/10, d1=0.836, d2=0.345 g=1.928
>66, 7/10, d1=0.345, d2=0.372 g=2.148
>66, 8/10, d1=0.594, d2=0.318 g=2.089
>66, 9/10, d1=0.736, d2=0.701 g=1.807
>66, 10/10, d1=0.110, d2=0.201 g=3.092
>67, 1/10, d1=0.567, d2=0.198 g=2.545
>67, 2/10, d1=0.337, d2=0.248 g=2.233
>67, 3/10

...

>230, 4/10, d1=0.822, d2=0.665 g=0.872
>230, 5/10, d1=0.583, d2=0.691 g=0.816
>230, 6/10, d1=0.606, d2=0.674 g=0.900
>230, 7/10, d1=0.811, d2=0.678 g=0.917
>230, 8/10, d1=0.669, d2=0.733 g=0.929
>230, 9/10, d1=0.680, d2=0.678 g=0.982
>230, 10/10, d1=0.815, d2=0.714 g=0.877
################# Summarize ###################
>Accuracy real: 33%, fake: 86%
>231, 1/10, d1=0.697, d2=0.666 g=0.908
>231, 2/10, d1=0.741, d2=0.627 g=0.934
>231, 3/10, d1=0.760, d2=0.667 g=0.956
>231, 4/10, d1=0.845, d2=0.680 g=0.931
>231, 5/10, d1=0.663, d2=0.679 g=0.827
>231, 6/10, d1=0.739, d2=0.690 g=0.931
>231, 7/10, d1=0.823, d2=0.683 g=0.829
>231, 8/10, d1=0.739, d2=0.738 g=0.844
>231, 9/10, d1=0.683, d2=0.699 g=0.889
>231, 10/10, d1=0.689, d2=0.793 g=0.911
>232, 1/10, d1=0.734, d2=0.657 g=0.893
>232, 2/10, d1=0.660, d2=0.783 g=0.889
>232, 3/10, d1=0.843, d2=0.709 g=0.837
>232, 4/10, d1=0.671, d2=0.704 g=0.839
>232, 5/10, d1=0.899, d2=0.686 g=0.846
>232, 6/10, d1=0.776, d2=0.664 g=0.880
>232, 7/10, d1=0.726, 

>250, 10/10, d1=0.761, d2=0.776 g=0.750
################# Summarize ###################
>Accuracy real: 47%, fake: 55%
>251, 1/10, d1=0.674, d2=0.673 g=0.778
>251, 2/10, d1=0.679, d2=0.760 g=0.817
>251, 3/10, d1=0.701, d2=0.688 g=0.824
>251, 4/10, d1=0.845, d2=0.836 g=0.760
>251, 5/10, d1=0.683, d2=0.804 g=0.827
>251, 6/10, d1=0.764, d2=0.670 g=0.792
>251, 7/10, d1=0.693, d2=0.708 g=0.892
>251, 8/10, d1=0.661, d2=0.720 g=0.881
>251, 9/10, d1=0.720, d2=0.758 g=0.894
>251, 10/10, d1=0.737, d2=0.700 g=0.784
>252, 1/10, d1=0.741, d2=0.724 g=0.727
>252, 2/10, d1=0.561, d2=0.746 g=0.764
>252, 3/10, d1=0.667, d2=0.729 g=0.833
>252, 4/10, d1=0.757, d2=0.622 g=0.799
>252, 5/10, d1=0.692, d2=0.776 g=0.752
>252, 6/10, d1=0.744, d2=0.770 g=0.749
>252, 7/10, d1=0.728, d2=0.834 g=0.789
>252, 8/10, d1=0.800, d2=0.698 g=0.758
>252, 9/10, d1=0.711, d2=0.789 g=0.713
>252, 10/10, d1=0.619, d2=0.779 g=0.779
>253, 1/10, d1=0.873, d2=0.670 g=0.781
>253, 2/10, d1=0.790, d2=0.747 g=0.734
>253, 3/10, d1=0.680,

>271, 4/10, d1=0.732, d2=0.859 g=0.707
>271, 5/10, d1=0.703, d2=0.761 g=0.745
>271, 6/10, d1=0.757, d2=0.776 g=0.747
>271, 7/10, d1=0.804, d2=0.810 g=0.760
>271, 8/10, d1=0.851, d2=0.842 g=0.715
>271, 9/10, d1=0.768, d2=0.810 g=0.768
>271, 10/10, d1=0.800, d2=0.797 g=0.755
>272, 1/10, d1=0.844, d2=0.796 g=0.732
>272, 2/10, d1=0.698, d2=0.811 g=0.698
>272, 3/10, d1=0.781, d2=0.795 g=0.720
>272, 4/10, d1=0.764, d2=0.808 g=0.728
>272, 5/10, d1=0.709, d2=0.785 g=0.736
>272, 6/10, d1=0.810, d2=0.825 g=0.748
>272, 7/10, d1=0.799, d2=0.724 g=0.774
>272, 8/10, d1=0.802, d2=0.751 g=0.759
>272, 9/10, d1=0.705, d2=0.866 g=0.733
>272, 10/10, d1=0.718, d2=0.757 g=0.722
>273, 1/10, d1=0.703, d2=0.757 g=0.778
>273, 2/10, d1=0.781, d2=0.762 g=0.803
>273, 3/10, d1=0.818, d2=0.727 g=0.744
>273, 4/10, d1=0.736, d2=0.746 g=0.757
>273, 5/10, d1=0.746, d2=0.742 g=0.766
>273, 6/10, d1=0.754, d2=0.754 g=0.705
>273, 7/10, d1=0.810, d2=0.750 g=0.755
>273, 8/10, d1=0.760, d2=0.779 g=0.715
>273, 9/10, d1=0.742, d

>291, 10/10, d1=0.711, d2=0.726 g=0.794
>292, 1/10, d1=0.708, d2=0.711 g=0.802
>292, 2/10, d1=0.647, d2=0.672 g=0.818
>292, 3/10, d1=0.772, d2=0.741 g=0.797
>292, 4/10, d1=0.704, d2=0.672 g=0.769
>292, 5/10, d1=0.710, d2=0.724 g=0.748
>292, 6/10, d1=0.718, d2=0.679 g=0.774
>292, 7/10, d1=0.609, d2=0.689 g=0.791
>292, 8/10, d1=0.681, d2=0.736 g=0.745
>292, 9/10, d1=0.721, d2=0.676 g=0.811
>292, 10/10, d1=0.636, d2=0.651 g=0.788
>293, 1/10, d1=0.729, d2=0.653 g=0.773
>293, 2/10, d1=0.712, d2=0.682 g=0.764
>293, 3/10, d1=0.718, d2=0.655 g=0.734
>293, 4/10, d1=0.591, d2=0.673 g=0.801
>293, 5/10, d1=0.673, d2=0.704 g=0.774
>293, 6/10, d1=0.828, d2=0.674 g=0.801
>293, 7/10, d1=0.699, d2=0.687 g=0.777
>293, 8/10, d1=0.745, d2=0.688 g=0.745
>293, 9/10, d1=0.672, d2=0.676 g=0.763
>293, 10/10, d1=0.725, d2=0.675 g=0.771
>294, 1/10, d1=0.719, d2=0.721 g=0.782
>294, 2/10, d1=0.713, d2=0.695 g=0.725
>294, 3/10, d1=0.733, d2=0.697 g=0.773
>294, 4/10, d1=0.745, d2=0.747 g=0.755
>294, 5/10, d1=0.672, 

>312, 6/10, d1=0.832, d2=0.686 g=0.737
>312, 7/10, d1=0.786, d2=0.696 g=0.724
>312, 8/10, d1=0.753, d2=0.753 g=0.735
>312, 9/10, d1=0.762, d2=0.716 g=0.712
>312, 10/10, d1=0.709, d2=0.745 g=0.700
>313, 1/10, d1=0.793, d2=0.791 g=0.679
>313, 2/10, d1=0.768, d2=0.747 g=0.691
>313, 3/10, d1=0.801, d2=0.754 g=0.697
>313, 4/10, d1=0.794, d2=0.777 g=0.698
>313, 5/10, d1=0.753, d2=0.745 g=0.681
>313, 6/10, d1=0.764, d2=0.747 g=0.714
>313, 7/10, d1=0.694, d2=0.826 g=0.690
>313, 8/10, d1=0.830, d2=0.779 g=0.719
>313, 9/10, d1=0.811, d2=0.809 g=0.693
>313, 10/10, d1=0.703, d2=0.800 g=0.728
>314, 1/10, d1=0.754, d2=0.705 g=0.701
>314, 2/10, d1=0.723, d2=0.746 g=0.716
>314, 3/10, d1=0.724, d2=0.785 g=0.737
>314, 4/10, d1=0.740, d2=0.776 g=0.716
>314, 5/10, d1=0.764, d2=0.751 g=0.728
>314, 6/10, d1=0.700, d2=0.798 g=0.667
>314, 7/10, d1=0.794, d2=0.844 g=0.713
>314, 8/10, d1=0.722, d2=0.758 g=0.735
>314, 9/10, d1=0.807, d2=0.749 g=0.708
>314, 10/10, d1=0.803, d2=0.776 g=0.694
>315, 1/10, d1=0.660, 